<a href="https://colab.research.google.com/github/dopey-tim/Bus4-118S/blob/main/prompt_engineering_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# AI Prompt Engineering Homework — Python Implementations

This notebook contains working Python examples for three prompt-engineering tasks:

1. **Prompt Chaining for a Customer Support AI** — a simple, rule-based chain that simulates a customer service flow and demonstrates how you'd wrap prompts if you were calling an LLM.
2. **Code Generation with ReACT Prompting** — a lightweight, illustrative ReACT-style loop that "reasons" (as printed logs), writes code, executes tests, and iterates if needed.
3. **Self-Reflection Prompt for Improving Output** — a reflective pass that critiques a draft summary and produces an improved version.

> **Note:** This notebook uses **no external APIs** by default. Where an LLM call would normally go, you'll find a `call_llm(...)` placeholder and a tiny `simulate_llm(...)` helper so the code runs out-of-the-box. If you want live LLM outputs, implement `call_llm(...)` with your preferred provider.


In [17]:
# 0) Make sure the SDK is current
!pip install -q --upgrade openai

# 1) Load key from Colab secrets or env
import os
try:
    from google.colab import userdata
    key = userdata.get("Tims-key")
    if key:
        os.environ["OPENAI_API_KEY"] = key
except Exception:
    pass
assert os.getenv("OPENAI_API_KEY"), "Missing OPENAI_API_KEY (or Colab secret 'Tims-key')."

# 2) Use Chat Completions
from typing import Optional
from openai import OpenAI

def call_llm(prompt: str, system: Optional[str] = None) -> str:
    client = OpenAI()  # reads OPENAI_API_KEY
    messages = []
    if system:
        messages.append({"role": "system", "content": system})
    messages.append({"role": "user", "content": prompt})
    resp = client.chat.completions.create(
        model="gpt-4o-mini",   # change to a model you have access to if needed
        messages=messages,
        temperature=0.2,
    )
    return resp.choices[0].message.content.strip()

# 3) Quick test
print(call_llm("Say 'pong' once.", system="You are terse."))


Pong.


In [22]:
from textwrap import dedent
import re
import traceback
from typing import Optional

# The call_llm function is now implemented in cell q-cAG46cZt81
# def call_llm(prompt: str, system: Optional[str] = None) -> str:
#     """
#     Placeholder for a real LLM call.
#     Implement with your provider of choice (e.g., OpenAI, Azure, etc.).
#     """
#     raise NotImplementedError("Plug your LLM API here if desired.")

def simulate_llm(prompt: str) -> str:
    lower = prompt.lower()

    # ... your other branches (intent/entity/next-step) ...

    # ReACT-ish code generation guidance (ensure this exact trigger)
    if "react" in lower and "write python" in lower and "two_sum" in lower:
        return dedent("""
        Thought: Use a hash map to find complements in O(n).
        Action: write_code
        Action Input:
        ```python
        def two_sum(nums, target):
            seen = {}
            for i, x in enumerate(nums):
                comp = target - x
                if comp in seen:
                    return [seen[comp], i]
                seen[x] = i
            return None
        ```
        """)
    # ⬇️ DEFAULT FALLBACK to avoid returning None
    return "Thought: Falling back.\nAction: write_code\nAction Input:\n```python\n# no-op\n```\n"


## 1) Prompt Chaining for a Customer Support AI

This example builds a simple multi-step chain:
1. **Intent Classification** → billing / account / technical / other  
2. **Entity Extraction** → order IDs, error codes, product names (toy)  
3. **Propose Next Step** → a short, helpful action suggestion  
4. **Final Response** → a customer-facing message using the chain outputs

You can replace `simulate_llm(...)` with `call_llm(...)` to swap in a real model.


In [19]:

from dataclasses import dataclass, field

@dataclass
class CSState:
    user_message: str
    intent: str | None = None
    entities: dict = field(default_factory=dict)
    next_step: str | None = None
    final_response: str | None = None

class CustomerSupportChain:
    def __init__(self, use_simulator: bool = True):
        self.use_sim = use_simulator

    def _ask_llm(self, prompt: str) -> str:
        if self.use_sim:
            return simulate_llm(prompt)
        return call_llm(prompt)

    def classify_intent(self, state: CSState) -> None:
        prompt = f"""
        Classify intent (billing | account | technical | other) for this user message.
        Return just the label.
        USER: {state.user_message}
        (Instruction: Classify intent)
        """
        state.intent = self._ask_llm(prompt).strip()

    def extract_entities(self, state: CSState) -> None:
        prompt = f"""
        Extract the key entities from the message: order IDs, error codes, product names.
        USER: {state.user_message}
        Intent: {state.intent}
        """
        raw = self._ask_llm(prompt)
        # quick parse for our simulator's "key=value" style
        entities = {}
        for part in raw.split(","):
            if "=" in part:
                k, v = part.split("=", 1)
                entities[k.strip()] = v.strip()
        state.entities = entities

    def propose_next_step(self, state: CSState) -> None:
        prompt = f"""
        Based on the intent "{state.intent}" and entities {state.entities},
        propose a concise next step for the agent.
        """
        state.next_step = self._ask_llm(prompt).strip()

    def craft_final_response(self, state: CSState) -> None:
        # A simple, friendly template
        state.final_response = dedent(f"""
        Thanks for reaching out! I see this looks like a **{state.intent}** request.
        {('I noted these details: ' + str(state.entities)) if state.entities else ''}

        **Next step:** {state.next_step}

        If that sounds good, reply **YES** to proceed, or tell me what to adjust.
        """).strip()

    def run(self, user_message: str) -> CSState:
        state = CSState(user_message=user_message)
        self.classify_intent(state)
        self.extract_entities(state)
        self.propose_next_step(state)
        self.craft_final_response(state)
        return state

# Demo
chain = CustomerSupportChain(use_simulator=False)

examples = [
    "I was double-charged on my invoice for the Pro Plan. Order #12345.",
    "I can't log in—password reset keeps failing for my Premium subscription.",
    "My headphones throw error code 0x501 and won't pair.",
    "Hi, I just have a general question about your shipping times."
]

for msg in examples:
    result = chain.run(msg)
    print('---')
    print('USER:', msg)
    print('INTENT:', result.intent)
    print('ENTITIES:', result.entities)
    print('NEXT STEP:', result.next_step)
    print('FINAL RESPONSE:\n', result.final_response)


---
USER: I was double-charged on my invoice for the Pro Plan. Order #12345.
INTENT: billing
ENTITIES: {}
NEXT STEP: Ask the customer for specific details about their billing inquiry, such as the type of billing issue they are experiencing or any relevant account information.
FINAL RESPONSE:
 Thanks for reaching out! I see this looks like a **billing** request.


**Next step:** Ask the customer for specific details about their billing inquiry, such as the type of billing issue they are experiencing or any relevant account information.

If that sounds good, reply **YES** to proceed, or tell me what to adjust.
---
USER: I can't log in—password reset keeps failing for my Premium subscription.
INTENT: technical
ENTITIES: {}
NEXT STEP: Identify the specific technical issue or topic that needs to be addressed and gather relevant information or resources to assist in resolving it.
FINAL RESPONSE:
 Thanks for reaching out! I see this looks like a **technical** request.


**Next step:** Identif


## 2) Code Generation with ReACT Prompting

We illustrate a minimal ReACT-style loop for a coding task. The "agent":

- Prints **Thought** → what to do next  
- Takes an **Action** → e.g., `write_code`  
- Gets an **Observation** → e.g., test results  
- Repeats until tests pass or attempts are exhausted

For demonstration, we solve `two_sum(nums, target)` using a hash map approach.  
Swap `simulate_llm(...)` with `call_llm(...)` to use a real model.


In [24]:
from textwrap import dedent
import re

def simulate_llm(prompt: str) -> str:
    """
    Tiny prompt-response simulator so the notebook runs without external APIs.
    """
    lower = prompt.lower()

    # Intent classification (very naive)
    if "classify intent" in lower:
        if any(k in lower for k in ["charge", "refund", "invoice", "billing"]):
            return "billing"
        if any(k in lower for k in ["can't log in", "password", "login", "signin", "sign in"]):
            return "account"
        if any(k in lower for k in ["error", "bug", "broken", "doesn't work", "crash", "not working", "501", "500", "404"]):
            return "technical"
        return "other"

    # Entity extraction (toy examples)
    if "extract the key entities" in lower or "extract entities" in lower:
        order = re.findall(r"(?:order|ticket)\s*#?\s*([a-z0-9\-]+)", prompt, flags=re.I)
        error = re.findall(r"(?:error(?:\s*code)?\s*[:#]?\s*)([A-Za-z0-9x\-]+)", prompt, flags=re.I)
        product = []
        if "pro plan" in lower: product.append("Pro Plan Subscription")
        if "premium" in lower: product.append("Premium Subscription")
        if "headphones" in lower: product.append("Headphones")
        if not order and "12345" in lower: order.append("12345")
        parts = []
        if order: parts.append(f"order_ids={order}")
        if error: parts.append(f"errors={error}")
        if product: parts.append(f"products={product}")
        return ", ".join(parts) if parts else "None found"

    # Resolution proposal (toy examples)
    if "propose a concise next step" in lower or "next step" in lower:
        if "billing" in lower:
            return "Offer a refund or billing adjustment and confirm the last 4 digits of the payment method."
        if "account" in lower:
            return "Send a secure password reset link and verify email ownership."
        if "technical" in lower:
            return "Ask for logs/screenshot, provide quick fixes, and offer to escalate a ticket."
        return "Ask for more details and route to the best team."

    # ReACT-ish code generation guidance (this is the Part 1 fix)
    if "react" in lower and "write python" in lower and "two_sum" in lower:
        return dedent("""
        Thought: Use a hash map to find complements in O(n).
        Action: write_code
        Action Input:
        ```python
        def two_sum(nums, target):
            seen = {}
            for i, x in enumerate(nums):
                comp = target - x
                if comp in seen:
                    return [seen[comp], i]
                seen[x] = i
            return None
        ```
        """)

    # Self-reflection critique
    if "critique this summary" in lower:
        issues = []
        if len(prompt.split()) < 80:
            issues.append("Too brief; lacks key details.")
        if "conclusion" not in lower:
            issues.append("No clear conclusion.")
        if "evidence" not in lower:
            issues.append("Missing supporting evidence.")
        return "Critique: " + (" ".join(issues) if issues else "Looks solid overall.")

    # Self-reflection revision
    if "rewrite an improved summary" in lower:
        return dedent("""
        Improved Summary: The article explains the core problem, outlines two viable solutions with trade-offs,
        and concludes by recommending the option with better long-term maintainability. It adds one concrete example
        and a brief note on limitations to increase credibility.
        """).strip()

    # ✅ DEFAULT FALLBACK (prevents None from being returned)
    return "Thought: Falling back.\nAction: write_code\nAction Input:\n```python\n# no-op\n```\n"


In [26]:
# Make sure these are imported once at top of your notebook
from textwrap import dedent
import re, traceback

class ReactCodeAgent:
    def __init__(self, use_simulator: bool = True, max_iters: int = 3):
        self.use_sim = use_simulator
        self.max_iters = max_iters

    def _ask_llm(self, prompt: str) -> str:
        resp = simulate_llm(prompt) if self.use_sim else call_llm(prompt)
        return resp if isinstance(resp, str) else ""

    def solve_two_sum(self):
        problem = dedent("""
        System: You are a helpful coding assistant.
        User: Use ReACT. Think step-by-step and write Python for a function:
        - Name: two_sum(nums, target)
        - Return the indices of two numbers such that they add up to target.
        - If none, return None.
        - Provide only valid Python inside a fenced code block.

        Follow this format:
        Thought: <your reasoning>
        Action: write_code
        Action Input:
        ```python
        # your code
        ```
        """)
        print("PROMPT SENT TO MODEL (abbrev):\n", problem.splitlines()[0], "...")

        ns, code = {}, ""
        for i in range(1, self.max_iters + 1):
            print(f"\n=== Iteration {i} ===")
            react = self._ask_llm(problem)
            if not isinstance(react, str) or not react.strip():
                print("Observation: Empty response from LLM. Retrying...")
                continue
            print(react.strip())

            # Extract code block
            code_match = re.search(r"```python(.*?)```", react, flags=re.S | re.I)
            if not code_match:
                print("Observation: No code found. Asking model to try again...")
                continue

            code = code_match.group(1).strip()
            print("Observation: Received code. Executing tests...")
            try:
                ns = safe_exec(code)
                tests = [
                    ("two_sum([2,7,11,15], 9)", [0,1]),
                    ("two_sum([3,2,4], 6)", [1,2]),
                    ("two_sum([3,3], 6)", [0,1]),
                    ("two_sum([1,2,3], 10)", None),
                ]
                results = run_tests(ns, tests)
                for line in results:
                    print(line)
                if all("PASS" in line for line in results):
                    print("\nFinal Answer: All tests passed ✅")
                    return ns, code
                else:
                    print("Thought: Some tests failed. I should revise the code.")
            except Exception:
                print("Observation: Execution error:\n", traceback.format_exc())
                print("Thought: Execution failed. I should fix syntax/logic and retry.")

        print("\nFinal Answer: Reached max iterations. Returning last attempt.")
        return ns, code




## 3) Self-Reflection Prompt for Improving Output

We take a **draft summary**, ask for a **critique**, and then request a **revision**.  
In a real setup you'd call your LLM twice; here we simulate both steps.


In [28]:
print(simulate_llm("critique this summary: test"))

Thought: Falling back.
Action: write_code
Action Input:
```python
# no-op
```



In [31]:
import inspect, textwrap
src = inspect.getsource(simulate_llm)
print(textwrap.dedent(src))


def simulate_llm(prompt: str) -> str:
    lower = prompt.lower()

    # ... your other branches (intent/entity/next-step) ...

    # ReACT-ish code generation guidance (ensure this exact trigger)
    if "react" in lower and "write python" in lower and "two_sum" in lower:
        return dedent("""
        Thought: Use a hash map to find complements in O(n).
        Action: write_code
        Action Input:
        ```python
        def two_sum(nums, target):
            seen = {}
            for i, x in enumerate(nums):
                comp = target - x
                if comp in seen:
                    return [seen[comp], i]
                seen[x] = i
            return None
        ```
        """)
    # ⬇️ DEFAULT FALLBACK to avoid returning None
    return "Thought: Falling back.\nAction: write_code\nAction Input:\n```python\n# no-op\n```\n"

    def __init__(self, use_simulator: bool = True, max_iters: int = 3):
        self.use_sim = use_simulator
        self.max_iters = max_it

In [30]:

def reflective_improvement(draft_summary: str) -> tuple[str, str]:
    critique_prompt = f"""
    Critique this summary for clarity, completeness, evidence, and conclusion.
    Be specific and concise. Then list 2-3 concrete improvements.

    SUMMARY:
    {draft_summary}
    """
    critique = simulate_llm(critique_prompt)

    improve_prompt = f"""
    Rewrite an improved summary that addresses the critique below.
    Keep it 4-6 sentences, specific, and self-contained.

    CRITIQUE:
    {critique}
    SUMMARY TO IMPROVE:
    {draft_summary}
    """
    improved = simulate_llm(improve_prompt)
    return critique.strip(), improved.strip()

# Demo
draft = (
    "The article talks about a new approach to system design. "
    "It briefly mentions trade-offs but doesn't go into much detail. "
    "There's no real conclusion."
)

critique, improved = reflective_improvement(draft)
print("=== Critique ===\n", critique, "\n")
print("=== Improved Summary ===\n", improved)


=== Critique ===
 Thought: Falling back.
Action: write_code
Action Input:
```python
# no-op
``` 

=== Improved Summary ===
 Thought: Falling back.
Action: write_code
Action Input:
```python
# no-op
```



### Notes & Next Steps

- To use a real model, implement `call_llm(...)` and switch `use_simulator=False` in the demos.
- You can extend the prompt chain with more steps (e.g., escalation, confirmation).
- The ReACT loop can be generalized to arbitrary coding tasks and richer toolsets.
- The reflection step is easy to wrap into any generation workflow as a second pass.
